# Torre de Hanoi

Pongamonos misticos:


> Cuenta la leyenda que unos brahmanes en un templo de Benarés han estado realizando el movimiento de la "Torre Sagrada de Brahma” sin parar desde hace siglos, la torre está formada por sesenta 
y cuatro discos de oro, y los movimientos obedecen a las siguientes místicas reglas:
> 1. Sólo se puede mover un disco a la vez.
> 2. Cada movimiento consiste en recoger el disco superior de una de las pilas y colocarlo encima de otra pila o sobre una varilla vacía.
> 3. Ningún disco podrá colocarse encima de un disco que sea más pequeño que él.
> 
> Una vez que finalicen la torre, va a llegar el fin del mundo.

La Torre de Hanói es un rompecabezas inventado en 1883 por el matemático francés **Édouard Lucas**. El rompecabezas comienza con los discos apilados en una varilla en orden de tamaño decreciente, 
el más pequeño en la parte superior, aproximándose así a una forma cónica. 

El objetivo del rompecabezas es mover toda la pila a una de las otras barras, con las reglas de la leyenda:
1. Sólo se puede mover un disco a la vez.
2. Cada movimiento consiste en coger el disco superior de una de las pilas y colocarlo encima de otra pila o sobre una varilla vacía.
3. Ningún disco podrá colocarse encima de un disco que sea más pequeño que él.

## Resolviendo este problema usando IA

Este problema es un típico problema para aplicar métodos de búsquedas. Podemos crear un agente que pueda resolver este problema. 

Limitemos a 5 discos, salvo que quieran usar 64 discos como los brahmanes.

El agente puede percibir cuantos discos y en qué orden hay en cada varilla. Además, puede tomar cualquier disco que se encuentre en la parte superior y moverlo a cualquier otra varilla que 
esté permitido moverlo. 

Definamos el problema para que podamos resolverlo,

### Espacio de estados:

Para 5 discos, tenemos $3^5 = 243$ posibles estados,

![estados_hanoi](./img/state_hanoi1.png)

### Estado inicial

Para este caso arrancamos con todos los discos de mayor a menor en la varilla izquierda.

![estados_hanoi_initial](./img/state_hanoi2.png)

### Estado objetivo

Para simplificar, vamos a tener un solo estado objetivo. Este caso el objetivo es terminar con todos los discos de mayor a menor en la varilla derecha.

![estados_hanoi_goal](./img/state_hanoi3.png)

----
Con la clase `StatesHanoi` podemos representar a un estado,

In [1]:
from hanoi_states import StatesHanoi

In [2]:
# Para representar la ubicación de los discos, usamos tres listas, uno por varilla, y un número del 1 al 5 para cada disco.

varilla_izquierda = [5, 4, 3, 2, 1]
varilla_medio = []
varilla_derecha = []

initial_state = StatesHanoi(varilla_izquierda, varilla_medio, varilla_derecha, max_disks=5)

varilla_izquierda = []
varilla_medio = []
varilla_derecha = [5, 4, 3, 2, 1]
goal_state = StatesHanoi(varilla_izquierda, varilla_medio, varilla_derecha, max_disks=5)

Con este estado implementado tenemos la posibilidad de imprimir el estado:

In [3]:
print(initial_state)

HanoiState: 5 4 3 2 1 |  | 


Y tenemos los siguientes métodos, entre varios más:

In [4]:
disk = initial_state.get_last_disk_rod(number_rod=0)
print(f"El ultimo disco de la varilla izquierda es {disk}")

if initial_state.check_valid_disk_in_rod(number_rod=1, disk=disk):
    print("Si, es posible poner el disco 1 en la varilla del medio?")
    
initial_state.put_disk_in_rod(number_rod=1, disk=disk)
print(f"El nuevo estado es: {initial_state}")

El ultimo disco de la varilla izquierda es 1
Si, es posible poner el disco 1 en la varilla del medio?
El nuevo estado es: HanoiState: 5 4 3 2 | 1 | 


In [5]:
varilla_izquierda = [5, 4, 3, 2, 1]
varilla_medio = []
varilla_derecha = []

initial_state = StatesHanoi(varilla_izquierda, varilla_medio, varilla_derecha, max_disks=5)

### Acciones

Además de tener los estados, tenemos las acciones que podemos aplicar para pasar de un estado a otro, es decir mover un disco de una varilla a otra,

In [6]:
from hanoi_states import ActionHanoi

In [7]:
action_example = ActionHanoi(disk=1, rod_input=0, rod_out=1)

# Movemos el disco de la varilla de la izquierda al disco del medio aplicando esta acción en el estado inicial
new_state = action_example.execute(state_hanoi=initial_state)

print(new_state)

HanoiState: 5 4 3 2 | 1 | 


Mover un disco de una varilla a otra, siempre que sea un movimiento permitido, cuesta lo mismo, que podemos definir como 1. Dado que arrancamos desde el estado inicial, y nos movemos al siguiente
estado, moviendo un disco, el nuevo estado va a tener un costo acumulado igual a 1:

In [8]:
print(f"El costo acumulado del nuevo estado es {new_state.accumulated_cost}")

El costo acumulado del nuevo estado es 1.0


### Problema de Hanoi

Por último podemos implementar el problema que tenga todo el problema incorporado, desde un estado iniciial, a un estado final, y la posibilidad de movimientos de un estado a otro.

In [9]:
from hanoi_states import ProblemHanoi

In [10]:
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

# Podemos ver todas las acciones que podemos aplicar desde un estado dado,
lista_acciones = problem.actions(new_state)
print(lista_acciones)

[Move disk 2 from 1 to 3, Move disk 1 from 2 to 1, Move disk 1 from 2 to 3]


Aplicamos una de las acciones que nos devuelve:

In [11]:
# Aplicamos la acción de Mover el disco 2 de 1 a 3
new_state_2 = problem.result(state=new_state, action=lista_acciones[0])

print(new_state_2)

HanoiState: 5 4 3 | 1 | 2


Podemos calcular el costo que nos llevo desde el estado inicial al que llegamos, como movimos dos discos es entonces el costo de 2:

In [12]:
problem.path_cost(c=1, state1=new_state, action=lista_acciones[0], state2=new_state_2)

2.0

Por último podemos ver si un estado dado es la solución:

In [13]:
if not problem.goal_test(state=new_state_2):
    print(f"{new_state_2} no es la solución final {goal_state}")

HanoiState: 5 4 3 | 1 | 2 no es la solución final HanoiState:  |  | 5 4 3 2 1


Con esta implementación ya tenemos la posibilidad de generar el grafo de estados de Hanoi,

![grafo_de_hanoi](./img/state_hanoi_graph.png)

----

## Algoritmos de búsqueda

Un algoritmo de búsqueda toma un problema de búsqueda como entrada y retorna una solución, o una indicación de falla.

La idea es buscar un camino que llegue al estado objetivo. Para ello vamos a construir un arbol que irá avanzando por estados del grafo hasta llegar al estado objetivo.  

![arbol_de_hanoi](./img/tree_hanoi.png)

Cada nodo del árbol corresponde a un **estado** y las aristas corresponde a una **acción**. Importante, el árbol **NO** es el grafo de estados. El grafo describe todo el set de estados, y 
las acciones que llevan de un lado a otro. El árbol describe el camino entre estos estados, para alcanzar el objetivo. 

Para poder aplicar los algoritmos, debemos definir la estructura de datos para hacer seguimiento del árbol. 

Los nodos del árbol son representados con los siguientes componentes:

- State: El estado, del espacio de estados, que corresponde el nodo.
- Node Parent: El nodo en el árbol de búsqueda que ha generado al nodo.
- Action: La acción que se aplicará al padre para generar el nodo.
- Path-Cost: El costo de un camino desde el nodo inicial al nodo.


In [14]:
from tree_hanoi import NodeHanoi

In [15]:
# Definimos la raíz del arbol
root = NodeHanoi(state=initial_state)

In [16]:
print("El arbol tiene como raíz a:")
print(root)

El arbol tiene como raíz a:
<Node HanoiState: 5 4 3 2 1 |  | >


Desde un nodo y definido el problema, podemos encontrar la frontera, la cual es la separación del grafo que ya fue explorada por el algoritmo de búsqueda y aquella que no.

![frontera_en_arbol_de_hanoi](./img/tree_hanoi_frontier.png)

In [17]:
# Expandimos la frontera del nodo raiz
lista_nodos_fronteras = root.expand(problem=problem)

In [18]:
# La raíz nos da dos nodos del arbol, uno con el disco 1 en la varilla del medio y otro con el disco 1 en la varilla de la derecha
lista_nodos_fronteras

[<Node HanoiState: 5 4 3 2 | 1 | >, <Node HanoiState: 5 4 3 2 |  | 1>]

Para expandir la frontera, necesitamos una estructura que nos ayude en esa tarea. Ya que en un algoritmo de búsqueda es vital seleccionar que nodo vamos a expandir primero. Como
vimos en clase, elegir el tipo de estructura para expandir es lo que define el tipo de algoritmo. 

La frontera se expande usando **colas**, la cual tenemos 3 tipos:

- Una cola **FIFO** (primero entra, primero sale) que toma los nodos en el mismo modo que se agregan.
- Una cola **LIFO** (último en salir, sale primero… o stack) quita el nodo más reciente.
- Una **cola prioritaria** que primer quita nodos con el mínimo costo de acuerdo con una función de evaluación f. 

Veamos por ejemplo, una implementación de cola FIFO:

In [19]:
# Implementamos usando una lista
fifo = []

La implementación debe incorporar la siguiente funciones

- **Add(Frontier)**: Inserta el nodo en su correspondiente lugar de la cola. En el caso de la FIFO, inserta a los nodos en la medida que van llegando

In [20]:
# Insertamos a los nodos que son frontera de la raíz en el orden que nos fue presentado:
fifo.insert(0, lista_nodos_fronteras[0])
fifo.insert(0, lista_nodos_fronteras[1])

print(fifo)

[<Node HanoiState: 5 4 3 2 |  | 1>, <Node HanoiState: 5 4 3 2 | 1 | >]


- **Is-empty(frontier)**: Retorna True si no hay nodos en la frontera. En el caso de la implementación de una lista, podemos preguntar si la cantidad de elemento es cero.

In [21]:
print("La cola está vacia?")
if len(fifo) == 0:
    print("La cola esta vacía")
else:
    print("La cola tiene elementos")

La cola está vacia?
La cola tiene elementos


- **Pop(frontier)**: Quita el primer nodo en la cola. Con las listas tenemos el método pop().

In [22]:
new_node = fifo.pop()
print(f"El nodo que sacamos es {new_node}")

El nodo que sacamos es <Node HanoiState: 5 4 3 2 | 1 | >


Podemos ver que una vez sacado el nodo, el mismo no está en la fila, y que particularmente sacamos el primer nodo que entró:

In [23]:
print("Los nodos que quedan en la fila son:")
print(fifo)

Los nodos que quedan en la fila son:
[<Node HanoiState: 5 4 3 2 |  | 1>]


Si expandimos la frontera del nuevo nodo que tenemos, podemos agregarlo a la cola:

In [24]:
lista_nodos_fronteras = new_node.expand(problem=problem)

# Insertamos a los nodos que son frontera de la raíz en el orden que nos fue presentado:
fifo.insert(0, lista_nodos_fronteras[0])
fifo.insert(0, lista_nodos_fronteras[1])
fifo.insert(0, lista_nodos_fronteras[2])

In [25]:
fifo

[<Node HanoiState: 5 4 3 2 |  | 1>,
 <Node HanoiState: 5 4 3 2 1 |  | >,
 <Node HanoiState: 5 4 3 | 1 | 2>,
 <Node HanoiState: 5 4 3 2 |  | 1>]

Podemos ver cual es nuestro siguiente nodo sin sacarlo (**Top(frontier)**: 

In [26]:
print(f"El siguiente nodo que podemos sacar es {fifo[-1]}")

El siguiente nodo que podemos sacar es <Node HanoiState: 5 4 3 2 |  | 1>


Nota: Este nodo es segundo que introducimos cuando introducimos la frontera de la raíz.

----

## Búsqueda primero en anchura

Implementamos un algoritmo de búsqueda con una estrategia sencilla. En esta se expande primero el nodo raíz, a continuación, se expanden todos los sucesores del nodo raíz, después sus sucesores, etc. 

![breadth_first_search](./img/breadth_first_search.png)


In [27]:
# Inicializaos el problema
initial_state = StatesHanoi([3, 2, 1], [], [], max_disks=3)
goal_state = StatesHanoi([], [], [3, 2, 1], max_disks=3)
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

frontier = [NodeHanoi(problem.initial)]  # Creamos una cola FIFO con el nodo inicial

# Mientras que la cola no este vacia
while len(frontier) != 0:
    node = frontier.pop()  # Extraemos el primer nodo de la cola
    if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
        last_node = node
        print("Encontramos la solución")
        break
    
    # Agregamos a la cola todos los nodos sucesores del nodo actual
    for next_node in node.expand(problem):
        frontier.insert(0, next_node)

Encontramos la solución


Este algoritmo lleva mucho tiempo, porque no estamos teniendo en cuenta si un nodo ya fue explorado, lo que nos hace demorar y entrar en bucles infinitos. En el caso de la torre 
de Hanoi, movernos de un estado a otro no posee dirección por lo que podemos ir de un estado a otro (por ejemplo, si tenemos el disco 1 en la varilla derecha, y la del medio está
vacía, podemos moverla a esta, pero una vez en este nuevo estado podemos volver la estado anterior y repetir indefinidamente). 

Entonces armemos una variante que guarde los nodos explorados, 

In [28]:
# Inicializaos el problema
initial_state = StatesHanoi([5, 4, 3, 2, 1], [], [], max_disks=5)
goal_state = StatesHanoi([], [], [5, 4, 3, 2, 1], max_disks=5)
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

frontier = [NodeHanoi(problem.initial)]  # Creamos una cola FIFO con el nodo inicial

explored = set()  # Este set nos permite ver si ya exploramos un estado para evitar repetir indefinidamente

# Mientras que la cola no este vacia
while len(frontier) != 0:
    node = frontier.pop()  # Extraemos el primer nodo de la cola
    
    # Agregamos nodo al set. Esto evita guardar duplicados, porque set nunca tiene elementos repetidos
    explored.add(node.state)
    
    if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
        last_node = node
        print("Encontramos la solución")
        break
    
    # Agregamos a la cola todos los nodos sucesores del nodo actual
    for next_node in node.expand(problem):
        # Solo si no fue explorado
        if next_node.state not in explored:
            frontier.insert(0, next_node)

Encontramos la solución


Una vez que encontramos la solución podemos ver, analizando el ultimo nodo del arbol, cuanto nos costo llegar a la solución:

In [29]:
print(f'Longitud del camino de la solución: {last_node.state.accumulated_cost}')

Longitud del camino de la solución: 31.0


Como todo paso tiene el mismo costo, la **búsqueda primero en achura** nos asegura que el camino más corto, como podemos ver.

Además, podemos ver cuantos nodos se explorados y cuantos nos quedaron en la cola,

In [30]:
print(len(explored), "nodos se expandieron y", len(frontier), "nodos quedaron en la frontera")

233 nodos se expandieron y 285 nodos quedaron en la frontera


Podemos ver el camino tomado que nos llevo a la solución, vamos a verlo al revez, desde el nodo final al nodo inicial:

In [31]:
node = last_node
while node.parent is not None:
    print(node.state)
    node = node.parent

HanoiState:  |  | 5 4 3 2 1
HanoiState: 1 |  | 5 4 3 2
HanoiState: 1 | 2 | 5 4 3
HanoiState:  | 2 1 | 5 4 3
HanoiState: 3 | 2 1 | 5 4
HanoiState: 3 | 2 | 5 4 1
HanoiState: 3 2 |  | 5 4 1
HanoiState: 3 2 1 |  | 5 4
HanoiState: 3 2 1 | 4 | 5
HanoiState: 3 2 | 4 1 | 5
HanoiState: 3 | 4 1 | 5 2
HanoiState: 3 | 4 | 5 2 1
HanoiState:  | 4 3 | 5 2 1
HanoiState: 1 | 4 3 | 5 2
HanoiState: 1 | 4 3 2 | 5
HanoiState:  | 4 3 2 1 | 5
HanoiState: 5 | 4 3 2 1 | 
HanoiState: 5 | 4 3 2 | 1
HanoiState: 5 2 | 4 3 | 1
HanoiState: 5 2 1 | 4 3 | 
HanoiState: 5 2 1 | 4 | 3
HanoiState: 5 2 | 4 1 | 3
HanoiState: 5 | 4 1 | 3 2
HanoiState: 5 | 4 | 3 2 1
HanoiState: 5 4 |  | 3 2 1
HanoiState: 5 4 1 |  | 3 2
HanoiState: 5 4 1 | 2 | 3
HanoiState: 5 4 | 2 1 | 3
HanoiState: 5 4 3 | 2 1 | 
HanoiState: 5 4 3 | 2 | 1
HanoiState: 5 4 3 2 |  | 1


Finalmente, si queremos medir tiempos y memoria, podemos hacer uso del metodo `%%timeit`

In [32]:
%%timeit 
# Inicializaos el problema
initial_state = StatesHanoi([5, 4, 3, 2, 1], [], [], max_disks=5)
goal_state = StatesHanoi([], [], [5, 4, 3, 2, 1], max_disks=5)
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

frontier = [NodeHanoi(problem.initial)]  # Creamos una cola FIFO con el nodo inicial

explored = set()  # Este set nos permite ver si ya exploramos un estado para evitar repetir indefinidamente

# Mientras que la cola no este vacia
while len(frontier) != 0:
    node = frontier.pop()  # Extraemos el primer nodo de la cola
    
    # Agregamos nodo al set. Esto evita guardar duplicados, porque set nunca tiene elementos repetidos
    explored.add(node.state)
    
    if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
        last_node = node
        break
    
    # Agregamos a la cola todos los nodos sucesores del nodo actual
    for next_node in node.expand(problem):
        # Solo si no fue explorado
        if next_node.state not in explored:
            frontier.insert(0, next_node)

45.2 ms ± 386 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Vemos que la solución demoro en promedio 45ms. 

En memoria, consume:

In [33]:
import tracemalloc

# Para medir memoria consumida (usamos el pico de memoria)
tracemalloc.start()

# Inicializaos el problema
initial_state = StatesHanoi([5, 4, 3, 2, 1], [], [], max_disks=5)
goal_state = StatesHanoi([], [], [5, 4, 3, 2, 1], max_disks=5)
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

frontier = [NodeHanoi(problem.initial)]  # Creamos una cola FIFO con el nodo inicial

explored = set()  # Este set nos permite ver si ya exploramos un estado para evitar repetir indefinidamente

# Mientras que la cola no este vacia
while len(frontier) != 0:
    node = frontier.pop()  # Extraemos el primer nodo de la cola
    
    # Agregamos nodo al set. Esto evita guardar duplicados, porque set nunca tiene elementos repetidos
    explored.add(node.state)
    
    if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
        last_node = node
        break
    
    # Agregamos a la cola todos los nodos sucesores del nodo actual
    for next_node in node.expand(problem):
        # Solo si no fue explorado
        if next_node.state not in explored:
            frontier.insert(0, next_node)
            
_, memory_peak = tracemalloc.get_traced_memory()
memory_peak /= 1024*1024
tracemalloc.stop()

print(f"Maxima memoria ocupada: {round(memory_peak, 2)} [MB]", )

Maxima memoria ocupada: 1.39 [MB]


Para terminar, si queremos exportar el camino para verlo en la animación, podemos crear los archivos haciendo:

In [34]:
last_node.generate_solution_for_simulator()

Para ver la simulación, se debe ejecutar el script en la carpeta `simulator`:

```bash
python3 ./simulator/simulation_hanoi.py
```

Una vez que se copiaron los archivos `initial_state.json` y `sequence.json` en la carpeta `simulator`. Recordar que se debe tener instalado en el entorno virtual a PyGame y 
Matplotlib. Se recomienda leer el archivo `README.md` que está en la carpeta `simulator`.